In [11]:
import os

In [12]:
%pwd

'D:\\data science and AI\\2. MLOPS COURSE BY KRISH\\14. First Project\\byme\\datascineceproject'

In [14]:
%pwd

'D:\\data science and AI\\2. MLOPS COURSE BY KRISH\\14. First Project\\byme\\datascineceproject'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
  root_dir: Path
  unzip_data_dir: Path
  STATUS_FILE: str
  all_schema: dict



In [16]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema

        )
        return data_validation_config

In [18]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile
import pandas as pd

In [22]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_cols_status = True
            validation_datatype_status=True
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()
            schema=self.config.all_schema
            
            for col in all_cols:
                if col not in all_schema:
                    validation_cols_status = False
                    break
                if str(data[col].dtype)!=str(schema[col]):
                    validation_datatype_status=False
                    break

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f'last validation column: {col}\n')
                f.write(f"Validation columns status: {validation_cols_status}\n")
                f.write(f"Validation datatype status: {validation_datatype_status}")
            
            return validation_cols_status and validation_datatype_status
        
        except Exception as e:
            raise e

    

In [23]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-09-04 15:24:34,231: INFO: common : created directory at: artifacts]
[2025-09-04 15:24:34,234: INFO: common : created directory at: artifacts/data_validation]
